In [8]:
from typing import Callable, Any, List
import os
import functools
import pickle

import xarray as xr
import numpy as np

from util import cache_to_file
from data import build_factor_data, build_factor_data2

In [11]:
halflifes = [21, 63, 126, 252]
factor_data = build_factor_data2(halflifes=halflifes)

ModuleNotFoundError: No module named 'numpy._core.numeric'

In [4]:
factor_data

<function data.build_factor_data(halflifes: List[int], factor_set='read', **kwargs) -> xarray.core.dataset.Dataset>

In [ ]:
# @cache_to_file
# def build_factor_data_cache(halflifes):
#     return build_factor_data2(halflifes)

In [ ]:
from datetime import datetime

today_date = datetime.today().date()
print(today_date)

In [ ]:
import pandas as pd

factor_data = build_factor_data_cache(halflifes=[63, 126, 252], read_cache=True)

date_latest = factor_data.indexes['date'].max()
date_today = pd.Timestamp(datetime.today().date())

# factor_data.indexes['date'].max() < datetime.today().date()


In [ ]:
def check_factor_data(factor_data: xr.Dataset) -> bool:
    date_latest = factor_data.indexes['date'].max()
    return date_latest >= date_today

In [ ]:
factor_data

In [ ]:
# def write_pickle(obj: Any, path: str) -> None:
#     with open(path, 'wb') as f:
#         pickle.dump(obj, f, protocol=-1)
        
# def read_pickle(path: str) -> Any:
#     with open(path, 'rb') as f:
#         return pickle.load(f)

In [ ]:
ttt_dir = 'cache'
ttt_file = 'data.pkl'

os.path.join(ttt_dir, ttt_file)

In [ ]:
build_factor_data_cache()

In [ ]:
build_factor_data_cache()

In [ ]:
def cache_decorator(func: Callable) -> Callable:
    """A decorator that caches function results to a Zarr file and Pickle file."""
    @functools.wraps(func)
    def wrapper(asset: List, read_cache: bool = True, write_cache: bool = True, cache_path: str = "cache", use_pickle: bool = False, **kwargs) -> Any:
        # Ensure cache directory exists
        os.makedirs(cache_path, exist_ok=True)
        
        # Define cache file paths
        pickle_cache_file = os.path.join(cache_path, "market_data.pkl")
        
        # Attempt to read from cache
        if read_cache:
            if use_pickle and os.path.exists(pickle_cache_file):
                with open(pickle_cache_file, "rb") as f:
                    return pickle.load(f)
            elif os.path.exists(zarr_cache_file):
                return xr.open_zarr(zarr_cache_file)
        
        # Call the original function
        result = func(asset, **kwargs)
        
        # Write result to cache
        if write_cache:
            if use_pickle:
                with open(pickle_cache_file, "wb") as f:
                    pickle.dump(result, f)
            else:
                result.to_zarr(zarr_cache_file, mode="w")
        
        return result
    
    return wrapper


In [ ]:
write_pickle(factor_data, 'factor_data.pkl')

In [ ]:
import zarr
factor_data.to_zarr('factor_data')

In [ ]:

def cache_decorator(func: Callable) -> Callable:
    """A decorator that caches function results to a Zarr file."""
    @functools.wraps(func)
    def wrapper(asset: List, read_cache: bool = True, write_cache: bool = True, cache_path: str = "cache", **kwargs) -> Any:
        # Ensure cache directory exists
        os.makedirs(cache_path, exist_ok=True)
        
        # Define cache file path
        cache_file = os.path.join(cache_path, "market_data.zarr")
        
        # Attempt to read from cache
        if read_cache and os.path.exists(cache_file):
            return xr.open_zarr(cache_file)
        
        # Call the original function
        result = func(asset, **kwargs)
        
        # Write result to cache
        if write_cache:
            result.to_zarr(cache_file, mode="w")
        
        return result
    
    return wrapper


In [ ]:

@cache_decorator
def get_market_data(asset: List) -> xr.Dataset:
    """Simulate fetching market data for a list of assets."""
    time = np.arange("2023-01-01", "2023-01-10", dtype="datetime64[D]")
    prices = np.random.rand(len(time), len(asset))
    return xr.Dataset({"price": ("time", "asset", prices)}, coords={"time": ("time", time), "asset": ("asset", asset)})


In [ ]:
# Example usage
assets = ["AAPL", "GOOG", "MSFT"]
data = get_market_data(assets, read_cache=True, write_cache=True)
print(data)